In [5]:
import os
import pandas as pd
import librosa
import numpy as np
import soundfile as sf
import random

# 配置文件路径
CONFIG = {
    "dataset_path": "../datasets/",
    "story_path": "../datasets/CBU0521DD_stories/",
    "label_path": "../datasets/CBU0521DD_stories_attributes.csv",
}

In [16]:
# 数据增强方法：时间拉伸、音高变换、添加噪音、改变音量
def time_stretch(y, rate=1.2):
    return librosa.effects.time_stretch(y, rate=rate)

def pitch_shift(y, sr, n_steps=4):
    return librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

def add_noise(y, noise_level=0.005):
    noise = np.random.randn(len(y))
    return y + noise_level * noise

def change_volume(y, factor=1.2):
    return y * factor

def process_audio(filename, story_path, sr=16000):
    y, _ = librosa.load(os.path.join(story_path, filename), sr=sr)
    
    augmented_audios = []
    augmented_audios.append(time_stretch(y))  # 时间拉伸
    augmented_audios.append(pitch_shift(y, sr))  # 音高变换
    augmented_audios.append(add_noise(y))  # 加入噪音
    augmented_audios.append(change_volume(y))  # 改变音量

    return [y] + augmented_audios

In [9]:
# 读取标签文件
label_df = pd.read_csv(CONFIG['label_path'])
filenames = label_df['filename'].tolist()

filenames[:5]

['00001.wav', '00002.wav', '00003.wav', '00004.wav', '00005.wav']

In [10]:
augmented_filenames = []
augmented_labels = []

# 增强后的音频存储路径
augmented_story_path = CONFIG['story_path'] + "_augmented/"
if not os.path.exists(augmented_story_path):
    os.makedirs(augmented_story_path)

In [17]:
# 遍历每个音频文件，进行数据增强
for filename in filenames:
    # 处理原始音频及其增强版本
    augmented_audios = process_audio(filename, CONFIG['story_path'])

    for i, y in enumerate(augmented_audios):
        augmented_filename = filename.split('.')[0] + f"_augmented_{i}.wav"
        augmented_filenames.append(augmented_filename)
        augmented_labels.append(label_df[label_df['filename'] == filename]['Story_type'].values[0])  # 保持标签不变

        # 保存增强后的音频
        sf.write(os.path.join(augmented_story_path, augmented_filename), y, 16000)

In [18]:
# 更新标签文件
augmented_labels_df = pd.DataFrame({
    "filename": augmented_filenames,
    "Language": ['Chinese'] * len(augmented_filenames),
    "Story_type": augmented_labels
})

# 保存新的标签文件
augmented_labels_df.to_csv(CONFIG['label_path'].replace('.csv', '_augmented.csv'), index=False)

print(f"数据增强完成，新的标签文件保存在：{CONFIG['label_path'].replace('.csv', '_augmented.csv')}")

数据增强完成，新的标签文件保存在：../datasets/CBU0521DD_stories_attributes_augmented.csv
